# Creating a Zero Trust Model for Microservices Architectures with SPIRE and Envoy - Lab 2
## Deploy the Microservices application in the Kubernetes Cluster with Envoy as a proxy to handle SPIFFE SVIDs

### **Lab workflow:**

In this lab:

1. As tenant user, you will first deploy the application services (Backend and Frontend applications) with their Envoy sidecar proxy, in the Kubernetes cluster in your tenant working context.

2. You will create registration entries on the SPIRE Server for your the Envoy proxy instances sitting in front of your application services.

3. You will explore the anatomy of the Envoy proxy configuration to learn how Envoy proxy retrieves the TLS elements, from the SPIRE system, that are required to establish mutual communication between workload services. 

4. You will then test successful X.509 authentication and mTLS connection between the Frontend app and the Backend app, through the Envoy sidecar proxies.

5. Next, you will configure an Envoy RBAC HTTP filter policy to allow for more granular access control.

6. Finally, you will perform some CleanUp to ensure repeatable hands-on workshop.

### **1- Initialize the environment**

Let's first define the environment variables needed to execute this part of the lab.

In [ ]:
#
# environment variables
#
studentId=$(grep hpecp-user $HOME/.kube/config | cut -d= -f2)
#
gateway_host="{{ HPEECPGWNAME }}"
Internet_access="{{ JPHOSTEXT }}"

BackendConfigMapUpdate="backend-envoy-configmap-update-student.yaml" #the backend envoy configmap object.
BackendConfigMapRBACUpdate="backend-envoy-configmap-update-RBAC-filter-student.yaml" #the backend envoy configmap object.

echo "Your studentId is: "$studentId

### **2- Deploy the Microservices application workloads in K8s cluster for your tenant working context**

To deploy the Backend and Frontent workload service, their Envoy sidecar proxies and configure the Envoy proxies to use SPIRE on behalf the workload services, you will leverage the Kubernetes ***Kustomize*** tool to customize Kubernetes resources through the following ***Kustomization*** Yaml file:
* Run the code cell below to display the content of the Kustomization YAML file:

In [ ]:
cat k8s-student/kustomization.yaml

Next, deploy and configure the workload services and their Envoy sidecar proxy by applying the following kubectl command:

In [ ]:
kubectl apply -k k8s-student/.

The kubectl apply command creates the following resources:

- A Kubernetes Deployment object for each of the workload services. The workload's POD contains a container for a particular service (Frontend, or Frontend-2, or Backend) and another container for the Envoy proxy.
- A Kubernetes Service object for each workload. It is used to communicate between them.
- Several Kubernetes Configmap objects are generated from various files (located at *~/k8s-student* subfolder) that are used to configure the workloads and the Envoy proxies sitting in front of the workloads:
   - ***json-data files*** are used to provide static data to the nginx instance running as the backend service.
   - ***envoy.yaml*** contains the Envoy proxy configuration for each workload.
   - ***symbank-webapp.conf*** contains the configuration supplied to each instance of the frontend services.

### **3- Inspect the deployed Microservices application**
Use the command `kubectl get -k k8s-student/.` and `kubectl get pods` to inspect the deployed components of your microservices application. Repeat the command until all the PODs are in **running** state.

In [ ]:
kubectl get pods | grep $studentId
kubectl get -k k8s-student/.

### **4- Create registration entries on the SPIRE Server for each of your Workload:**

In order for SPIRE to identify a workload service and issue the service identity document (the X.509 certificate SVID) to the service, you must **register** the service with the SPIRE Server, via registration entries. Workload service registration tells SPIRE how to identify the service and which service identifier (SPIFFE ID) and certificate SVID (which carries the encoded/cryptographic form of the SPIFFE ID) to give it. 

A registration entry maps an identity - in the form of a SPIFFE ID - to a set of properties known as **selectors** (for example: namespace, POD service account, POD label, container name) and that are attributes the workload service must possess in order to be issued a particular service identity (the SPIFFE ID and the X.509 certificate SVID). A workload service registration entry contains the following:

* a SPIFFE ID that uniquely identifies a workload service and takes the following format: _spiffe://trust-domain/workload-identifier_
* a set of one or more selectors such as the K8s namespace and the POD Service Account
* a parent ID (this is the SPIFFE ID assigned to the SPIRE Agent running on K8S worker nodes)

In our use case, the SPIFFE ID granted to the workload service is derived from the POD Service Account and namespace in the form:
  
  ***spiffe://Trusted-domain/ns/namespace/sa/Pod-sa/app-service-name*** 

An example of registration entry for the ***backend*** application service with a POD name *backend-student75*, an Envoy proxy container named *envoy*, a POD Service Account *student75* in the *testspiresds* namespace, in SPIFFE trusted domain *example.org* is shown here:

***
    -parentID spiffe://example.org/ns/spire/sa/spire-agent \
    -spiffeID spiffe://example.org/ns/testspiresds/sa/student75/backend \
    -selector k8s:ns:testspiresds \
    -selector k8s:sa:student75 \
    -selector k8s:pod-label:app:backend-student75 \
    -selector k8s:container-name:envoy
***

> <font color="red"> Note: the selectors for our workloads point to the Envoy container: **k8s:container-name:envoy**. This is how we configure Envoy to perform X.509 SVID authentication on a workload’s behalf. </font>
   

Let’s use the following Bash script to create the registration entries on SPIRE Server for your three workloads. 

In [ ]:
bash Scripts/create-registration-entries-student.sh

Once the script is run, the list of created registration entries can be shown using the command below: 

In [ ]:
echo "Listing created registration entries..."
kubectl exec -n spire spire-server-0 -- /opt/spire/bin/spire-server entry show | grep -B 1 -C 5 -E $studentId

Upon successful creation of the registration entries for the workload services, all the service SVIDs are generated and signed on the SPIRE Server.

The SPIRE server then sends to the SPIRE agent running on the K8s worker nodes a list of all registration entries for workload services that are entitled to run on the nodes. SPIRE agents cache these registration entries. 

The Workload services (in our case the Envoy proxies) connect to the SPIRE agent using the Workload API &mdash; exposed by the SPIRE agent &mdash; to retrieve their service identifier (SPIFFE ID), their identity service document (the X.509 certificate SVID) and the Trust Bundle. When a service connects to the SPIRE Agent to retrieve its SVID, the SPIRE agent performs a process called ***Workload Attestation*** to determine the service's identity and return the correct SVID to the service.
During the Workload Attestation process, the SPIRE agent determines the workload service's identity and which SVID to assign to the service by comparing the attributes the workload service possesses to registration entries selectors.

>**Note: In our use case, the Envoy proxies connect to the Workload API, retrieve their SPIFFE ID, SVIDs and Trust Bundle, and encrypt and authenticate traffic on behalf of the Frontend/Backend workload services.**

### **5- Anatomy of the Envoy sidecar proxy configuration**
To help you better understand how Envoy sidecar proxy obtain their service identities to encrypt mTLS communication on workload service's behalf, let's examine some of the sections of the **Envoy.yaml** file used to create the Kubernetes configMap object for  the configuration of the Envoy proxy container of the workload services.

#### **5.1. Envoy proxy to SPIRE communication and TLS context**

A common configuration of the Envoy proxy sitting in front of each of the workload services is composed of two major elements:

A Unix Domain Socket and a TLS context.

Let's explore the configuration for these two elements in the configMap objects you have just deployed on the Kubernetes cluster for the backend and frontend workload services.

##### **5.1.1. Unix Domain Socket:**

An Envoy proxy must be configured to communicate with the SPIRE agent by configuring a **cluster** that points to the Unix Domain Socket the SPIRE agent provides on the Kubernetes worker node the Envoy proxy container runs.

<p align="center">
  <img src="Pictures/Envoy-Spire-cluster.png">
    
>A **Unix Domain Socket** is an inter-process communicaion mechanism that allows bidirectional communication between processes running on the same machine. You can list a machine local unix domain sockets with the following command: *netstat -a -p --unix | grep spire*. An example of the output of the command on a Kubernetes worker node where a SPIRE agent is running is shown here:
    
<p align="center">
  <img src="Pictures/Spire-agent-local-unix-domain-socket.png">

##### **5.1.2. TLS Context:**

We then configure a **TLS context** that is used by the Envoy proxy to retrieve its SPIFFE ID, certificate SVID and the Trust Bundle that will allow it to encrypt and authenticate traffic on behalf of the workload service.

The ***tls_certificate_sds_secret_configs*** configuration tells Envoy proxy to talk to the SPIRE agent over the Unix Domain Socket to get its service identity document (SVID). The name of the TLS certificate is the SPIFFE ID of the workload service that Envoy is acting as a proxy for. Since the SPIFFE ID (_spiffe://trust-domain/workload-identifier_) that uniquely identifies the workload service has been created in SPIRE, the Envoy sidecar proxy will be able to get its SVID from SPIRE agent. The picture below is an excerpt from the Envoy proxy sitting in front of the **backend** service. 

<p align="center">
  <img src="Pictures/Envoy-TLS_Context.png">
    

Next, the **combined validation context** will verify the trust domain as well as the Subject Alternate Name (SAN) of the peer service. 

The ***match_subject_alt_names*** of the *default_validation_context* configuration tells Envoy to only allow the connection if the certificate SVID presented by the connection has a Subject Alternate Name (SAN) that matches a particular workload service SPIFFE ID. 

In this example below, the Envoy sidecar proxy of the **Backend** workload will only allow connection from the Frontend and Frontend-2 workloads.

The ***validation_context_sds_secret_config*** tells Envoy proxy to talk to the SPIRE agent over the Unix Domain Socket to get the Trust Bundle which contains the CA certificates that Envoy uses to verify peer certificate SVID when establishing encrypted and authenticated connection with peer services.

<p align="center">
  <img src="Pictures/Envoy-combined-validation-context.png">
    

##### **5.1.3. Envoy inbound and outbound traffic:**

Envoy proxy sidecar configuration can be categorized in **listeners** and **clusters**.

- **Listeners** configure the **inbound** traffic. This includes routing of inbound traffic from different addresses, ports, URLs to the intended cluster sections. The **TLS context** of a **listener** allows the Envoy proxy to secure the corresponding **inbound** traffic with **mTLS** as described above.

- **Clusters** configure the **outbound** traffic, traffic from the Envoy proxy to any other address and port or socket. As for listeners, the **TLS context** of a **cluster** allows the Envoy proxy to secure the corresponding **outbound** traffic with **mTLS** as described above.

You will see in the following steps that Envoy proxies communicating mTLS secured with each other always have a pair of **cluster** and **listener** both with a configured **TLS context**.

#### **5.2. How Mutual TLS communication between Frontend application and Backend application works through the Envoy proxies**

All of the Envoy proxies form a communication mesh in which each application sends and receives messages to and from **localhost** and is unaware of the network topology. 

>**Note:** Remember, the deployed workload service and their Envoy sidecar proxy are containers running on the same Kubernetes POD resource. Containers in the same POD share the same resources and local network. Containers can communicate with other containers in the same POD as though they were on the same machine (i.e.: localhost).

##### **5.2.1. Frontend web app service configuration:**

In [ ]:
cat k8s-student/frontend/config/symbank-webapp.conf

In our use case, the file **symbank-webapp.conf** contains the static configuration supplied to each instance of the frontend service. With this configuration, the Frontend web application will connect to the listining socket on port 3001 on localhost, that is the Envoy proxy sitting in front of the Frontend application.

<p align="center">
  <img src="Pictures/Frontend-config-webapp.png">

---

##### **5.2.2. Frontend Envoy proxy configuration:**

In [ ]:
kubectl describe configmap frontend-envoy-$studentId



The Envoy proxy for a Frontend application is configured:

* to listen on localhost on port 3001 by configuring a **Listener**, and
* to communicate with Envoy proxy sitting in front of the Backend application service by configuring a **cluster** that points to Kubernetes service deployed for the Backend's Envoy proxy that listens on port 9001 and directs all requests on port 9001 to the Envoy proxy.

<p align="center">
  <img src="Pictures/Envoy-Frontend-config-Listener.png">
    

<p align="center">
  <img src="Pictures/Envoy-Frontend-config-for-cluster-backend.png">

In this case, as we want the communication between Frontend Envoy sidecar and Backend Envoy sidecar to be secured with SPIRE and mTLS, the **tls_context** is configured for the frontend **cluster** section pointing to the backend Envoy sidecar. This results in the frontend envoy proxy requesting an SVID from SPIRE for all ***outbound traffic*** for this cluster pointing to the backend proxy including identifying itself with its SPIFFE ID and encryption with TLS.

<p align="center">
  <img src="Pictures/Envoy-Frontend-config-for-TLS-Context.png">

---

##### **5.2.3. Backend Envoy proxy configuration:**

In [ ]:
kubectl describe configmap backend-envoy-$studentId

The Envoy proxy for the Backend application is configured:
* to listen on Localhost on port 9001 by configuring a **Listener**, and
* to communicate with the Backend application service by configuring a **cluster** that points to the localhost on port 80 
    
<p align="center">
  <img src="Pictures/Envoy-Backend-config-Listener.png">
     
<p align="center">
  <img src="Pictures/Envoy-Backend-config-Local-service-cluster.png">


In this case, to secure the communication between Frontend Envoy sidecar and Backend Envoy sidecar SPIRE and mTLS, the **tls_context** is configured in the backend **listener** section. This results in the backend envoy proxy requesting an SVID from SPIRE for all ***inbound traffic*** for this listener including identifying itself with its SPIFFE ID and encryption with TLS.

<p align="center">
  <img src="Pictures/Envoy-Backend-config-for-TLS-Context.png">

### **6- Test the workloads authentication and mutual TLS secure communication**
Now that your application workloads have been deployed, registered in SPIRE Server, and their Envoy sidecar proxy have obtained their service identity document (X.509 SVID), the Envoy sidecar proxies can use their certificate SVID to establish end-to-end mutual TLS encrypted connections on application workload's behalf and secure connections between the applications.

Now let’s test the authorization that we’ve configured. To do this, we show that both frontend applications (frontend and frontend-2) can talk to the backend service through their Envoy proxies.

* Run the code cell below to obtain your Frontend application services endpoints and connect to them from your browser. 
* Once the page is loaded for the Frontend application, you will see the bank account and transaction details for user ***"Jacob Marley"***.
* Connect to the Frontend-2 application and you will see the account and transaction details for user ***"Alex Fergus"***.

> <font color="red"> **Note:** If you are seeing a security warning on your browser about the certificate while connecting to the Frontend applications, please accept the risk and proceed to continue.</font>


In [ ]:
#
# Frontend app endpoint URL:
#
FrontendAppURL=$(kubectl describe service frontend-${studentId} | grep gateway/3000 | awk '{print $3}')
#echo $FrontendAppURL
FrontendAppPort=$(echo $FrontendAppURL | cut -d':' -f 2)
#echo $FrontendAppPort
echo "Your Frontend application service endpoint URL is: "https://$Internet_access:$FrontendAppPort
#
# Frontend-2 app endpoint URL:
#
Frontend2AppURL=$(kubectl describe service frontend-2-${studentId} | grep gateway/3002 | awk '{print $3}')
#echo $Frontend2AppURL
Frontend2AppPort=$(echo $Frontend2AppURL | cut -d':' -f 2)
#echo $Frontend2AppPort
echo "Your Frontend-2 application service endpoint URL is: "https://$Internet_access:$Frontend2AppPort

### **7- Update the TLS configuration so only one Frontend app can access the backend**

The Envoy configuration for the backend service uses the TLS configuration to filter incoming connections by validating the Subject Alternative Name (SAN) of the certificate presented on the TLS connection. For SVIDs, the SAN field of the certificate is set with the SPIFFE ID associated with the service. So by specifying the SPIFFE IDs in the _match_subject_alt_names_ filter we indicate to Envoy which services can establish a connection.

Let’s now update the Envoy configuration for the backend service to revoke the workload entry for the Frontend-2 workload, and allow requests from the frontend service only. This is achieved by removing the SPIFFE ID of the frontend-2 service from the _combined_validation_context_ section at the Envoy configuration. 

To update the Envoy configuration for the backend workload, you will use the file ***backend-envoy-configmap-update-student.yaml*** to update the configMap for the Envoy proxy of the Backend workload service.

After execution of the code cell below, check out the section ***Combined_validation_context*** of the TLS_Context section, and run the next cell to apply the new configuration on Envoy sidecar proxy of the Backend workload:

In [ ]:
cat $BackendConfigMapUpdate

In [ ]:
kubectl apply -f $BackendConfigMapUpdate

Next, the backend pod needs to be restarted for the Envoy sidecar proxy container to pick up the new configuration:

In [ ]:
kubectl scale deployment backend-${studentId} --replicas=0
kubectl scale deployment backend-${studentId} --replicas=1

### <font color="red">Wait some seconds for the deployment to propagate before trying to view the frontend-2 service in your browser again.</font> 

In [ ]:
kubectl get pod | grep ${studentId}

Once the pod is ready, **refresh** the browser using the correct URL (run the code cell below to get the URL) for frontend-2 service. As a result, now Envoy does not allow the request to get to the backend service and account details are not shown in your browser.

In [ ]:
echo "Your Frontend-2 application service is available at endpoint URL here: "https://$Internet_access:$Frontend2AppPort

On the other hand, you can check that the frontend service is still able to get a response from the backend. **Refresh** the browser at the correct URL (run the code cell below to get the URL) and confirm that account details are shown for **Jacob Marley**.

In [ ]:
echo "Your Frontend application service is available at endpoint URL here: "https://$Internet_access:$FrontendAppPort

### **8- Extend the scenario with a Role Based Access Control (RBAC) filter**

Envoy provides a Role Based Access Control (RBAC) HTTP filter that checks the request based on a list of policies. A policy consists of permissions and principals, where the principal specifies the downstream client identities of the request, for example, the URI SAN of the downstream client certificate. So we can use the SPIFFE ID assigned to the service to create policies that allow for more granular access control.

The Symbank demo application consumes three different endpoints to get all the information about the bank account. The _/profiles_ endpoint provides the name and the address of the account’s owner. The other two endpoints, _/balances_ and _/transactions_, provide the balance and transactions for the account.

To demonstrate an Envoy RBAC filter, we can create a policy that allows the **Frontend** service to obtain only data from the _/profile_ endpoint and deny requests sent to other endpoints. This is achieved by defining a policy with a principal that matches the SPIFFE ID of the service and the permissions to allow only GET requests to the _/profiles_ resource. 

Let’s now update the Envoy proxy configuration for the backend service to allow requests from the frontend service to obtain only data from the _/profile_ only. 

To update the Envoy configuration for the backend workload, you will use the file ***backend-envoy-configmap-update-RBAC-filter-student.yaml*** to update the configMap for the Envoy proxy of the Backend workload service.

After execution of the code cell below, look at the section ***http_filters*** for configuration details of the RBAC filter, and run the next cell to apply the new configuration on Envoy sidecar proxy of the Backend workload:

In [ ]:
cat $BackendConfigMapRBACUpdate

In [ ]:
kubectl apply -f $BackendConfigMapRBACUpdate

Next, the backend pod needs to be restarted for the Envoy sidecar proxy container to pick up the new configuration:

In [ ]:
kubectl scale deployment backend-${studentId} --replicas=0
kubectl scale deployment backend-${studentId} --replicas=1

### <font color="red">Wait some seconds for the deployment to propagate before trying to view the frontend-2 service in your browser again.</font> 

In [ ]:
kubectl get pod | grep ${studentId}

Once the pod is ready, **refresh** the browser using the correct URL (run the code cell below to get the URL) for Frontend service and confirm that **only** the profile information for **Jacob Marley's** account is shown.

As a result, now Envoy does not allow the request to get account details beyond the Profile information.

In [ ]:
echo "Your Frontend application service is available at endpoint URL here: "https://$Internet_access:$FrontendAppPort

### **9- Time to do some Cleanup**

Please make sure you run the code cell below to delete your workloads from the Kubernetes cluster and delete their registration entries from SPIRE Server.  

In [ ]:
kubectl delete -k k8s-student/.

In [ ]:
bash Scripts/delete-registration-entries-student.sh

Verifying there is no registration entry for your workloads. The output should be ***":1"*** that means there is no more entries registered in SPIRE Server for your workloads.

In [ ]:
echo "Listing created registration entries..."
kubectl exec -n spire spire-server-0 -- /opt/spire/bin/spire-server entry show | grep -B 1 -C 5 -E $studentId

## Summary

In this lab, you went through the steps to configure SPIRE to provide service identity dynamically in the form of X.509 certificates that are consumed by Envoy secret discovery service (SDS), so workloads can communicate securely.

## Let's move to the conclusion now!

* [Conclusion](3-WKSHP-Conclusion.ipynb)